# What is PySpark?
pyspark is a python api for working with apache spark. I will first explain what do I mean by a "python api" for something and then explain what, specifically, is 'apache spark'.

what I mean by **'python api'** is that you can use the syntex and agility of python to interact with and send commands to a system that is not based, at its core, on python. 

with pyspark, you intercat with apache spark - a system designed for working, analyzing and modeling with immense amounts of data in many computers at the same time. putting it in a different way, apache spark allows you to run computations in parallel, instead of sequentially. it allows you to divide one incredibly large task into many smaller tasks, and run each such task on a different machine.this allowes you to accomplish your analysis goals in reasonable time that would not be possible on a single machine.

usually, we would define the amount of data that suits PySpark as what would not fit into a single machine storage (let alone RAM).

**important related concepts:** 
1. distributed computing - when you distribute a task into several smaller task that run at the same time. this is what pyspark allows you to do with many machines, but it can also be done on a single machine with several threads, for example.
2. cluster - a network of machines that can take on tasks from a user, interact with one another and return results. these provide the computing resources that pyspark will use to make the computations.
3. Resilient Distributed Dataset (RDD) - an immutable distributed collection of data. it is not tabular, like DataFrames which we will work with later, and has no data schema. therefore, for tabular data wrangling, DataFrames allowes for more API options and uner-then-hood optimizations. still, you might encounter RDDs as you learn more about Spark, and should be aware of their existence.

**Part of PySpark we will cover:**
1. PySpark SQL - contains commands for data processing and manipulation.
2. PySpark MLlib - includes a variety of models, model training and related commands.

**Spark Architecture:**
to send commands and receive results from a cluster, you will need to initiate a spark session. this object is your tool for interacting with Spark. each user of the cluster will have its own Spark Session, that will allow him to use the cluster in isolation from other users. all of the sessions are communicating with a spark context, which is the master node in the cluster - that is, it assigns each of computers in the cluster tasks and coordinates them. each of the computers in the cluster that perform tasks for a master node is called a worker node. to connect to a worker node, the master node needs to get that node's comput power allocated to it, by a cluster manager, that is responsable for distributing the cluster resources. inside each worker node, there are execute programs that run the tasks - they can run multiple tasks simultaneously, and has their own cashe for storing results. so, each master node can have multiple worker nodes, that can have multiple tasks running.  

In [1]:
# a SparkSession object can perform the most common data processing tasks
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('test').getOrCreate() # will return existing session if one was
                                                           # created before and was not closed

23/10/05 18:10:41 WARN Utils: Your hostname, EliteX2 resolves to a loopback address: 127.0.1.1; using 10.1.8.69 instead (on interface wifi0)
23/10/05 18:10:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/05 18:10:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

**dataset:**
https://www.kaggle.com/fedesoriano/heart-failure-prediction

In [3]:
# read csv, all columns will be of type string
df = spark.read.option('header','true').csv('heart.csv')
# tell pyspark the type of the columns - saves time on large dataset. there are other ways to do this, but that's
# my favorite
schema = 'Age INTEGER, Sex STRING, ChestPainType STRING'
df = spark.read.csv('heart.csv', schema=schema, header=True)
# let PySpark infer the schema
df = spark.read.csv('heart.csv', inferSchema=True, header=True)
# replace nulls with other value at reading time
df = spark.read.csv('heart.csv', nullValue='NA')
# save data
df.write.format("csv").save("heart_save.csv")
# if you want to overwrite the file
df.write.format("csv").mode("overwrite").save("heart_save.csv")

In [5]:
df = spark.read.csv('heart.csv', inferSchema=True, header=True)


In [6]:
# show head of table
df.show(3)

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 52|  1|  0|     125| 212|  0|      1|    168|    0|    1.0|    2|  2|   3|     0|
| 53|  1|  0|     140| 203|  1|      0|    155|    1|    3.1|    0|  0|   3|     0|
| 70|  1|  0|     145| 174|  0|      1|    125|    1|    2.6|    0|  0|   3|     0|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
only showing top 3 rows



In [7]:
# count number of rows
df.count()

1025

In [9]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- cp: integer (nullable = true)
 |-- trestbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalach: integer (nullable = true)
 |-- exang: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: integer (nullable = true)
 |-- ca: integer (nullable = true)
 |-- thal: integer (nullable = true)
 |-- target: integer (nullable = true)



In [11]:
# show parts of the table
df.select('Age').show(3)
df.select('Age','Sex').show(3)

+---+
|Age|
+---+
| 52|
| 53|
| 70|
+---+
only showing top 3 rows

+---+---+
|Age|Sex|
+---+---+
| 52|  1|
| 53|  1|
| 70|  1|
+---+---+
only showing top 3 rows



## Pandas DataFrame VS PySpark DataFrame

both represents a table of data with rows and columns. however, under the hood they are different, as PySpark dataframe needs to support distributed computations. as we move forward, we will see more and more features of it that are not present in Pandas DataFrame. that being said - if you know how to use Pandas, than moving to PySpark will feel like a natural transition.

## DAG
directed acyclic graph is the way Spark runs computations. when you give it a series of transformation to apply to the dataset, it build a graph out of those transformations, so it knows what to do - but it does not execute those commands immediately, if it does not have to. rather, it is lazy - it will go through the DAG and apply the transformations only when it must, to provide a needed result. this allows better performance, since spark knows what's ahead of a certain computation and get optimize the process accordingly.

## transformations VS actions
in PySpark, there are two types of command: transformations and actions. transformation commands are added to the DAG, but does not get it to actually be executed. they transform one DataFrame into another, not changing the input DataFrame. on the other hand, actions make PySpark execute the DAG but does not create a new DataFrame - instead, they output the result of the DAG.

## Caching
every time you run a DAG, it will be re-computed from the beginning. that is, the results are not saved in memory. 
so, if we want to save a result so it won't have to be recomputed, we can use the cache command. note, that this will occupy space in the working node's memory - so be careful with the sizes of datasets you are caching! by default, the cached DF is stored to RAM, and is unserialized (not converted into a stream of bytes). you can change both of these - store data to hard disk, serialized it, or both!

## Collecting
even after caching a DataFrame, it still sits in the worker nodes memory. if you want to collect is pieces, assemble them and save them on the master node so you won't have to pull it every time, use the command for collecting. again, be very careful with this, since the collected file will have to fit in the master node memory!

In [14]:
df.collect()

[Row(age=52, sex=1, cp=0, trestbps=125, chol=212, fbs=0, restecg=1, thalach=168, exang=0, oldpeak=1.0, slope=2, ca=2, thal=3, target=0),
 Row(age=53, sex=1, cp=0, trestbps=140, chol=203, fbs=1, restecg=0, thalach=155, exang=1, oldpeak=3.1, slope=0, ca=0, thal=3, target=0),
 Row(age=70, sex=1, cp=0, trestbps=145, chol=174, fbs=0, restecg=1, thalach=125, exang=1, oldpeak=2.6, slope=0, ca=0, thal=3, target=0),
 Row(age=61, sex=1, cp=0, trestbps=148, chol=203, fbs=0, restecg=1, thalach=161, exang=0, oldpeak=0.0, slope=2, ca=1, thal=3, target=0),
 Row(age=62, sex=0, cp=0, trestbps=138, chol=294, fbs=1, restecg=1, thalach=106, exang=0, oldpeak=1.9, slope=1, ca=3, thal=2, target=0),
 Row(age=58, sex=0, cp=0, trestbps=100, chol=248, fbs=0, restecg=0, thalach=122, exang=0, oldpeak=1.0, slope=1, ca=0, thal=2, target=1),
 Row(age=58, sex=1, cp=0, trestbps=114, chol=318, fbs=0, restecg=2, thalach=140, exang=0, oldpeak=4.4, slope=0, ca=3, thal=1, target=0),
 Row(age=55, sex=1, cp=0, trestbps=160, c

In [15]:
# convert PySpark DataFrame to Pandas DataFrame
pd_df = df.toPandas()
# convert it back
spark_df = spark.createDataFrame(pd_df)

In [16]:
# show first three rows as three row objects, which is how spark represents single rows from a table.
# we will learn more about it later
df.head(3)

[Row(age=52, sex=1, cp=0, trestbps=125, chol=212, fbs=0, restecg=1, thalach=168, exang=0, oldpeak=1.0, slope=2, ca=2, thal=3, target=0),
 Row(age=53, sex=1, cp=0, trestbps=140, chol=203, fbs=1, restecg=0, thalach=155, exang=1, oldpeak=3.1, slope=0, ca=0, thal=3, target=0),
 Row(age=70, sex=1, cp=0, trestbps=145, chol=174, fbs=0, restecg=1, thalach=125, exang=1, oldpeak=2.6, slope=0, ca=0, thal=3, target=0)]

In [17]:
# type os columns
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- cp: integer (nullable = true)
 |-- trestbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalach: integer (nullable = true)
 |-- exang: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: integer (nullable = true)
 |-- ca: integer (nullable = true)
 |-- thal: integer (nullable = true)
 |-- target: integer (nullable = true)



In [23]:
# column dtypes as list of tuples
df.dtypes

[('Age', 'float'),
 ('sex', 'int'),
 ('cp', 'int'),
 ('trestbps', 'int'),
 ('chol', 'int'),
 ('fbs', 'int'),
 ('restecg', 'int'),
 ('thalach', 'int'),
 ('exang', 'int'),
 ('oldpeak', 'double'),
 ('slope', 'int'),
 ('ca', 'int'),
 ('thal', 'int'),
 ('target', 'int')]

In [24]:
# cast a column from one type to other
from pyspark.sql.types import FloatType
df = df.withColumn("Age",df.Age.cast(FloatType()))
df = df.withColumn("RestingBP",df.Age.cast(FloatType()))

In [26]:
# compute summery statistics
df.select(['Age','RestingBP']).describe()

DataFrame[summary: string, Age: string, RestingBP: string]

In [27]:
df.select(['Age','RestingBP']).describe().show()

+-------+-----------------+-----------------+
|summary|              Age|        RestingBP|
+-------+-----------------+-----------------+
|  count|             1025|             1025|
|   mean|54.43414634146342|54.43414634146342|
| stddev|9.072290233244278|9.072290233244278|
|    min|             29.0|             29.0|
|    max|             77.0|             77.0|
+-------+-----------------+-----------------+



In [28]:
# add a new column or replace existing one
AgeFixed = df['Age'] + 1  # select alwayes returns a DataFrame object, and we need a column object
df = df.withColumn('AgeFixed', AgeFixed)

In [29]:
df.select(['AgeFixed','Age']).describe().show()

+-------+-----------------+-----------------+
|summary|         AgeFixed|              Age|
+-------+-----------------+-----------------+
|  count|             1025|             1025|
|   mean|55.43414634146342|54.43414634146342|
| stddev|9.072290233244278|9.072290233244278|
|    min|             30.0|             29.0|
|    max|             78.0|             77.0|
+-------+-----------------+-----------------+



In [33]:
# remove columns
df.drop('AgeFixed').show(1) # add df = to get the new DataFrame into a variable

+----+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+---------+
| Age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|RestingBP|
+----+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+---------+
|52.0|  1|  0|     125| 212|  0|      1|    168|    0|    1.0|    2|  2|   3|     0|     52.0|
+----+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+---------+
only showing top 1 row



In [34]:
# rename a column
df.withColumnRenamed('Age','age').select('age').show(1)
# to rename more than a single column, i would suggest a loop.
name_pairs = [('Age','age'),('Sex','sex')]
for old_name, new_name in name_pairs:
    df = df.withColumnRenamed(old_name,new_name)

+----+
| age|
+----+
|52.0|
+----+
only showing top 1 row



In [35]:
df.select(['age','sex']).show(1)

+----+---+
| age|sex|
+----+---+
|52.0|  1|
+----+---+
only showing top 1 row



In [36]:
# drop all rows that contain any NA
df = df.na.drop()
df.count()
# drop all rows where all values are NA
df = df.na.drop(how='all')
# drop all rows where more at least 2 values are NOT NA
df = df.na.drop(thresh=2)
# drop all rows where any value at specific columns are NAs.
df = df.na.drop(how='any', subset=['age','sex']) # 'any' is the defult

In [37]:
# fill missing values in a specific column with a '?'
df = df.na.fill(value='?',subset=['sex'])
# replace NAs with mean of column
from pyspark.ml.feature import Imputer # In statistics, imputation is the process of
                                       # replacing missing data with substituted values
imptr = Imputer(inputCols=['age','RestingBP'],
                outputCols=['age','RestingBP']).setStrategy('mean') # can also be 'median' and so on

df = imptr.fit(df).transform(df)

In [40]:
# filter to adults only and calculate mean
df.filter('age > 18')
df.where('age > 18')# 'where' is an alias to 'filter'
df.where(df['age'] > 18) # third option
# add another condition ('&' means and, '|' means or)
df.where((df['age'] > 18) | (df['cp'] == 'ATA'))
# take every record where the 'ChestPainType' is NOT 'ATA'
df.filter(~(df['cp'] == 'ATA'))

DataFrame[age: float, sex: int, cp: int, trestbps: int, chol: int, fbs: int, restecg: int, thalach: int, exang: int, oldpeak: double, slope: int, ca: int, thal: int, target: int, RestingBP: float, AgeFixed: float]

In [41]:
df.filter('age > 18').show()

+----+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+---------+--------+
| age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|RestingBP|AgeFixed|
+----+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+---------+--------+
|52.0|  1|  0|     125| 212|  0|      1|    168|    0|    1.0|    2|  2|   3|     0|     52.0|    53.0|
|53.0|  1|  0|     140| 203|  1|      0|    155|    1|    3.1|    0|  0|   3|     0|     53.0|    54.0|
|70.0|  1|  0|     145| 174|  0|      1|    125|    1|    2.6|    0|  0|   3|     0|     70.0|    71.0|
|61.0|  1|  0|     148| 203|  0|      1|    161|    0|    0.0|    2|  1|   3|     0|     61.0|    62.0|
|62.0|  0|  0|     138| 294|  1|      1|    106|    0|    1.9|    1|  3|   2|     0|     62.0|    63.0|
|58.0|  0|  0|     100| 248|  0|      0|    122|    0|    1.0|    1|  0|   2|     1|     58.0|    59.0|
|58.0|  1|  0|     114| 318|  0|      2|    140|    0|    4.4|  

In [42]:
# evaluate a string expression into command
from pyspark.sql.functions import expr
exp = 'age + 0.2 * AgeFixed'
df.withColumn('new_col', expr(exp)).select('new_col').show(3)

+-------+
|new_col|
+-------+
|   62.6|
|   63.8|
|   84.2|
+-------+
only showing top 3 rows



In [45]:
# group by age
disease_by_age = df.groupby('age').mean().select(['age','avg(trestbps)'])


In [46]:
# sort values in desnding order
from pyspark.sql.functions import desc
disease_by_age.orderBy(desc("age")).show(5)

+----+------------------+
| age|     avg(trestbps)|
+----+------------------+
|77.0|             125.0|
|76.0|             140.0|
|74.0|             120.0|
|71.0|124.36363636363636|
|70.0|146.28571428571428|
+----+------------------+
only showing top 5 rows



In [48]:
from pyspark.sql.functions import asc
disease_by_age = df.groupby('age').mean().select(['age','avg(trestbps)'])
disease_by_age.orderBy(desc("age")).show(3)

+----+-------------+
| age|avg(trestbps)|
+----+-------------+
|77.0|        125.0|
|76.0|        140.0|
|74.0|        120.0|
+----+-------------+
only showing top 3 rows



In [49]:
# aggregate to get several statistics for several columns
# the available aggregate functions are avg, max, min, sum, count
from pyspark.sql import functions as F
df.agg(F.min(df['age']),F.max(df['age']),F.avg(df['sex'])).show()

+--------+--------+------------------+
|min(age)|max(age)|          avg(sex)|
+--------+--------+------------------+
|    29.0|    77.0|0.6956097560975609|
+--------+--------+------------------+



In [29]:
df.groupby('HeartDisease').agg(F.min(df['age']),F.avg(df['sex'])).show()

+------------+--------+--------+--------+
|HeartDisease|min(age)|max(age)|avg(sex)|
+------------+--------+--------+--------+
|           1|      31|      77|    null|
|           0|      28|      76|    null|
+------------+--------+--------+--------+



In [53]:
# run an SQL query on the data
df.createOrReplaceTempView("df") # tell PySpark how the table will be called in the SQL query
spark.sql("""SELECT sex from df""").show(2)

# we also choose columns using SQL sytnx, with a command that combins '.select()' and '.sql()'
df.selectExpr("age >= 40 as older", "age").show(2)

+---+
|sex|
+---+
|  1|
|  1|
+---+
only showing top 2 rows

+-----+----+
|older| age|
+-----+----+
| true|52.0|
| true|53.0|
+-----+----+
only showing top 2 rows



In [55]:
df.createOrReplaceTempView("df2") 
spark.sql("""SELECT sex from df2""").show(2)

+---+
|sex|
+---+
|  1|
|  1|
+---+
only showing top 2 rows



In [61]:
df.groupby('age', 'sex').count().show(3000)

+----+---+-----+
| age|sex|count|
+----+---+-----+
|68.0|  0|    3|
|77.0|  1|    3|
|43.0|  1|   19|
|48.0|  0|    3|
|62.0|  1|   13|
|76.0|  0|    3|
|46.0|  1|   14|
|56.0|  0|   10|
|35.0|  1|   11|
|43.0|  0|    7|
|74.0|  0|    3|
|54.0|  1|   38|
|58.0|  1|   47|
|51.0|  1|   27|
|68.0|  1|    9|
|64.0|  1|   25|
|66.0|  0|   11|
|49.0|  0|    6|
|39.0|  0|    7|
|69.0|  1|    6|
|70.0|  1|   14|
|51.0|  0|   12|
|52.0|  1|   40|
|52.0|  0|    3|
|67.0|  0|    9|
|41.0|  0|   12|
|53.0|  0|    9|
|40.0|  1|   11|
|54.0|  0|   15|
|34.0|  1|    3|
|35.0|  0|    4|
|62.0|  0|   24|
|55.0|  0|   15|
|57.0|  1|   45|
|44.0|  0|    6|
|34.0|  0|    3|
|69.0|  0|    3|
|50.0|  0|    9|
|59.0|  0|    3|
|61.0|  0|    8|
|48.0|  1|   20|
|37.0|  1|    3|
|71.0|  0|   11|
|42.0|  0|    6|
|44.0|  1|   30|
|57.0|  0|   12|
|41.0|  1|   20|
|60.0|  1|   24|
|58.0|  0|   21|
|37.0|  0|    3|
|45.0|  0|   10|
|61.0|  1|   23|
|65.0|  0|   13|
|59.0|  1|   43|
|45.0|  1|   15|
|56.0|  1|   2

In [62]:
df.groupby('age').pivot('sex').count().show(10000)

+----+----+----+
| age|   0|   1|
+----+----+----+
|64.0|   9|  25|
|47.0|null|  18|
|58.0|  21|  47|
|39.0|   7|   7|
|41.0|  12|  20|
|29.0|null|   4|
|67.0|   9|  22|
|49.0|   6|  11|
|56.0|  10|  29|
|54.0|  15|  38|
|55.0|  15|  15|
|38.0|null|  12|
|51.0|  12|  27|
|48.0|   3|  20|
|66.0|  11|  14|
|70.0|null|  14|
|65.0|  13|  14|
|43.0|   7|  19|
|53.0|   9|  17|
|68.0|   3|   9|
|40.0|null|  11|
|77.0|null|   3|
|57.0|  12|  45|
|35.0|   4|  11|
|34.0|   3|   3|
|71.0|  11|null|
|50.0|   9|  12|
|76.0|   3|null|
|69.0|   3|   6|
|44.0|   6|  30|
|42.0|   6|  20|
|45.0|  10|  15|
|62.0|  24|  13|
|46.0|   9|  14|
|52.0|   3|  40|
|59.0|   3|  43|
|63.0|  17|  15|
|60.0|  13|  24|
|61.0|   8|  23|
|37.0|   3|   3|
|74.0|   3|null|
+----+----+----+



In [63]:
# pivot - expensive operation
df.selectExpr("age >= 40 as older", "age",'sex').groupBy("sex")\
                    .pivot("older", ("true", "false")).count().show()

+---+----+-----+
|sex|true|false|
+---+----+-----+
|  1| 673|   40|
|  0| 295|   17|
+---+----+-----+



In [42]:
df.select(['age','MaxHR','Cholesterol']).show(4)

+---+-----+-----------+
|age|MaxHR|Cholesterol|
+---+-----+-----------+
| 40|  172|        289|
| 49|  156|        180|
| 37|   98|        283|
| 48|  108|        214|
+---+-----+-----------+
only showing top 4 rows



In [72]:
df.printSchema()

root
 |-- age: float (nullable = true)
 |-- sex: integer (nullable = true)
 |-- cp: integer (nullable = true)
 |-- trestbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalach: integer (nullable = true)
 |-- exang: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: integer (nullable = true)
 |-- ca: integer (nullable = true)
 |-- thal: integer (nullable = true)
 |-- target: integer (nullable = true)
 |-- RestingBP: float (nullable = true)
 |-- AgeFixed: float (nullable = true)
 |-- Fvec: vector (nullable = true)



In [89]:
# devide dataset to training features and target
X_column_names = ['age','slope','oldpeak']
target_colum_name = ['target']

# convert feature columns into a columns where the vlues are feature vectors

X = df.select(['age','slope','oldpeak'])
X.show(3)

+----+-----+-------+
| age|slope|oldpeak|
+----+-----+-------+
|52.0|    2|    1.0|
|53.0|    0|    3.1|
|70.0|    0|    2.6|
+----+-----+-------+
only showing top 3 rows



In [93]:
Y = df.select('target')

In [90]:
# devide dataset into training and testing sets
trainset, testset = X.randomSplit([0.8,0.2])

In [94]:
# devide dataset into training and testing sets
ytrain, ytest = Y.randomSplit([0.8,0.2])

In [95]:
trainset.show(5)

+----+-----+-------+
| age|slope|oldpeak|
+----+-----+-------+
|29.0|    2|    0.0|
|29.0|    2|    0.0|
|29.0|    2|    0.0|
|34.0|    2|    0.0|
|34.0|    2|    0.0|
+----+-----+-------+
only showing top 5 rows



In [96]:
# predict 'RestingBP' using linear regression
from pyspark.ml.regression import LinearRegression
model = LinearRegression()
model = model.fit(trainset, ytrain)


TypeError: Params must be either a param map or a list/tuple of param maps, but got <class 'pyspark.sql.dataframe.DataFrame'>.

In [48]:
# evaluate model
model.evaluate(testset).predictions.show(3)

+---+-----------+------------+-----+------------------+
|Age|Cholesterol|        Fvec|MaxHR|        prediction|
+---+-----------+------------+-----+------------------+
| 28|        132|[28.0,132.0]|  185|159.46829176931507|
| 30|        237|[30.0,237.0]|  170|162.32394700931022|
| 34|        210|[34.0,210.0]|  192|157.08382627073425|
+---+-----------+------------+-----+------------------+
only showing top 3 rows



In [97]:
# handel categorical features with ordinal indexing
from pyspark.ml.feature import StringIndexer
indxr = StringIndexer(inputCol='ChestPainType', outputCol='ChestPainTypeInxed')
indxr.fit(df).transform(df).select('ChestPainTypeInxed').show(3)

Py4JJavaError: An error occurred while calling o924.fit.
: org.apache.spark.SparkException: Input column ChestPainType does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:293)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:293)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:290)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	at org.apache.s